## Question 1
Compute the average temperature by season ('season_desc'). (The temperatures are numbers between 0 and 1, but don't worry about that. Let's say that's the Shellman temperature scale.)

I get
```
season_desc
Fall           0.711445
Spring         0.321700
Summer         0.554557
Winter         0.419368
```
Which clearly looks wrong. Figure out what's wrong with the original data and fix it.

##Question 2
Various of the columns represent dates or datetimes, but out of the box pd.read_table won't treat them correctly. This makes it hard to (for example) compute the number of rentals by month. Fix the dates and compute the number of rentals by month.

##Question 3
Investigate how the number of rentals varies with temperature. Is this trend constant across seasons? Across months?

##Question 4
There are various types of users in the usage data sets. What sorts of things can you say about how they use the bikes differently?